# Milestone 4a: Data Preprocessing
> Author: Tien Ly  
> Created: July 2024

## Objective
The objective of this notebook (a part of milestone 4 of my internship at NOAA PolarWatch) is to preprocess sea ice thickness (SIT) data and integrate it with atmospheric and oceanic variables to enhance an attention-based LSTM model for predicting monthly sea ice extent with a 1-month lead time. For reference, the model being replicated is based on the journal article [here](https://s3.us-east-1.amazonaws.com/climate-change-ai/papers/icml2021/50/paper.pdf), and the full GitHub repository for this project can be found [here](https://github.com/big-data-lab-umbc/sea-ice-prediction/tree/main/climate-change-ai-workshop).


### Dataset used
Sea Ice Thickness and Multi-variable Extended AVHRR Polar Pathfinder APP-X NCEI Climate Data
Record V2, Arctic, 1982-Present, Twice Daily
https://polarwatch.noaa.gov/erddap/griddap/ncei_polarAPPX20_nhem.graph


## Import libraries

In [105]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime

## Part 1: Load Sea Ice Thickness Data

In [106]:
# Get the twice-daily SIT data from ERDDAP
# Open the dataset in xarray
url = 'https://polarwatch.noaa.gov/erddap/griddap/ncei_polarAPPX20_nhem'
ds = xr.open_dataset(url)
ds

Note:Caching=1


<xarray.Dataset> Size: 309GB
Dimensions:                                    (time: 29643, rows: 361,
                                                columns: 361)
Coordinates:
  * time                                       (time) datetime64[ns] 237kB 19...
  * rows                                       (rows) float32 1kB -4.5e+06 .....
  * columns                                    (columns) float32 1kB -4.525e+...
Data variables: (12/20)
    cdr_sea_ice_thickness                      (time, rows, columns) float32 15GB ...
    cdr_surface_temperature                    (time, rows, columns) float32 15GB ...
    cdr_surface_albedo                         (time, rows, columns) float32 15GB ...
    cdr_surface_downwelling_shortwave_flux     (time, rows, columns) float32 15GB ...
    cdr_surface_downwelling_longwave_flux      (time, rows, columns) float32 15GB ...
    cdr_surface_upwelling_shortwave_flux       (time, rows, columns) float32 15GB ...
    ...                                         ...
    cloud_optical_depth                        (time, rows, columns) float32 15GB ...
    cloud_top_pressure                         (time, rows, columns) float32 15GB ...
    cloud_top_temperature                      (time, rows, columns) float32 15GB ...
    cloud_type                                 (time, rows, columns) float32 15GB ...
    surface_shortwave_cloud_radiative_forcing  (time, rows, columns) float32 15GB ...
    surface_longwave_cloud_radiative_forcing   (time, rows, columns) float32 15GB ...
Attributes: (12/44)
    _NCProperties:              version=2,netcdf=4.8.1,hdf5=1.10.6
    acknowledgement:            Please acknowledge NCEI, CoastWatch West Coas...
    cdm_data_type:              Grid
    cdr_program:                NOAA Climate Data Record Program for satellites
    cdr_variable:               cdr_surface_temperature, cdr_surface_albedo, ...
    comment:                    In order to be compliant with the EASE Grid s...
    ...                         ...
    spatial_resolution:         25 km
    standard_name_vocabulary:   CF Standard Name Table (v26, 08 November 2013)
    summary:                    summary
    time_coverage_end:          2024-08-05T14:00:00Z
    time_coverage_start:        1982-01-01T04:00:00Z
    title:                      Sea Ice Thickness and Multi-variable Extended...

In [7]:
# Extract the SIT data from the dataset
da_sit = ds['cdr_sea_ice_thickness']
da_sit

<xarray.DataArray 'cdr_sea_ice_thickness' (time: 29643, rows: 361, columns: 361)> Size: 15GB
[3863105403 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 237kB 1982-01-01T04:00:00 ... 2024-08-05T1...
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
Attributes:
    _ChunkSizes:            [  1 361 361]
    colorBarPalette:        KT_ice
    coverage_content_type:  physicalMeasurement
    grid_mapping:           crs
    ioos_category:          Ice Distribution
    long_name:              NOAA CDR of sea ice thickness
    standard_name:          sea_ice_thickness
    units:                  m
    valid_range:            [ 0. 10.]

### Print out some useful metadata

In [8]:
print('Dimensions:', list(da_sit.dims))
print('Coordinates:', list(da_sit.coords))

Dimensions: ['time', 'rows', 'columns']
Coordinates: ['time', 'rows', 'columns']


In [9]:
print('Earliest date:', da_sit.time.values[0])
print('Most recent date:', da_sit.time.values[-1])

Earliest date: 1982-01-01T04:00:00.000000000
Most recent date: 2024-08-05T14:00:00.000000000


In [11]:
print("Is xgrid's first value -->", round(da_sit.rows[0].item(), 6))
print('greater than') 
print("xgrid's last value -->", round(da_sit.rows[-1].item(), 6))

print(da_sit.rows[0].item() > da_sit.rows[-1].item())

Is xgrid's first value --> -4499620.5
greater than
xgrid's last value --> 4524688.5
False


In [12]:
print("Is ygrid's first value -->", round(da_sit.columns[0].item(), 6))
print('greater than') 
print("ygrid's last value -->", round(da_sit.columns[-1].item(), 6))

print(da_sit.columns[0].item() > da_sit.columns[-1].item())

Is ygrid's first value --> -4524688.5
greater than
ygrid's last value --> 4499620.5
False


## Part 2: Compute the Monthly Climatology (1982-2018)

In [107]:
# Extract the dataset for the climatological period: 1982-2018
start_date = '1982-01-01'
end_date = '2018-12-31'
da_sit_clim = da_sit.sel(time=slice(start_date, end_date))
da_sit_clim

<xarray.DataArray 'cdr_sea_ice_thickness' (time: 26714, rows: 361, columns: 361)> Size: 14GB
[3481395194 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 214kB 1982-01-01T04:00:00 ... 2018-12-31T1...
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
Attributes:
    _ChunkSizes:            [  1 361 361]
    colorBarPalette:        KT_ice
    coverage_content_type:  physicalMeasurement
    grid_mapping:           crs
    ioos_category:          Ice Distribution
    long_name:              NOAA CDR of sea ice thickness
    standard_name:          sea_ice_thickness
    units:                  m
    valid_range:            [ 0. 10.]

In [108]:
# Create a range of monthly periods across 1982-2018
monthly_periods = pd.date_range(start=start_date, end=end_date, freq='MS')
monthly_periods

DatetimeIndex(['1982-01-01', '1982-02-01', '1982-03-01', '1982-04-01',
               '1982-05-01', '1982-06-01', '1982-07-01', '1982-08-01',
               '1982-09-01', '1982-10-01',
               ...
               '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
               '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
               '2018-11-01', '2018-12-01'],
              dtype='datetime64[ns]', length=444, freq='MS')

### Compute monthly means in batches

In [35]:
monthly_means = []

# Iterate over the first 100 monthly periods and calculate means
for i in range(0, 100):
    # Define the current month
    month_start = monthly_periods[i]
    month_end = monthly_periods[i + 1] - pd.DateOffset(days=1)
    
    # Extract the current month
    da_sit_monthly = da_sit_clim.sel(time=slice(month_start, month_end))
    
    # Calculate the monthly mean
    monthly_mean = da_sit_monthly.mean(dim='time')
    monthly_means.append(monthly_mean)

print(len(monthly_means))
print(monthly_means[0])

100
<xarray.DataArray 'cdr_sea_ice_thickness' (rows: 361, columns: 361)> Size: 521kB
array([[ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)
Coordinates:
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06


In [36]:
# Repeat for the next 100 periods
for i in range(100, 200):
    month_start = monthly_periods[i]
    month_end = monthly_periods[i + 1] - pd.DateOffset(days=1)
    
    da_sit_monthly = da_sit_clim.sel(time=slice(month_start, month_end))
    
    monthly_mean = da_sit_monthly.mean(dim='time')
    monthly_means.append(monthly_mean)

print(len(monthly_means))

200


In [37]:
# Repeat for the next 100 periods
for i in range(200, 300):
    month_start = monthly_periods[i]
    month_end = monthly_periods[i + 1] - pd.DateOffset(days=1)
    
    da_sit_monthly = da_sit_clim.sel(time=slice(month_start, month_end))
    
    monthly_mean = da_sit_monthly.mean(dim='time')
    monthly_means.append(monthly_mean)

print(len(monthly_means))

300


In [38]:
# Continue for the next 100 periods
for i in range(300, 400):
    month_start = monthly_periods[i]
    month_end = monthly_periods[i + 1] - pd.DateOffset(days=1)
    
    da_sit_monthly = da_sit_clim.sel(time=slice(month_start, month_end))
    
    monthly_mean = da_sit_monthly.mean(dim='time')
    monthly_means.append(monthly_mean)

print(len(monthly_means))

400


In [39]:
# Complete the remaining periods
for i in range(400, len(monthly_periods)):
    month_start = monthly_periods[i]
    if i == len(monthly_periods) - 1: # handle last period separately
        month_end = pd.to_datetime(datetime.date(2018, 12, 31))
    else:
        month_end = monthly_periods[i + 1] - pd.DateOffset(days=1)
    
    da_sit_monthly = da_sit_clim.sel(time=slice(month_start, month_end))
    
    monthly_mean = da_sit_monthly.mean(dim='time')
    monthly_means.append(monthly_mean)

print(len(monthly_means))

444


### Concatenate and save data

In [43]:
# Concatenate the monthly means list into a single DataArray
final_monthly_means = xr.concat(monthly_means, dim='time')
# Assign the time coordinate to the DataArray
final_monthly_means = final_monthly_means.assign_coords(time=monthly_periods)
final_monthly_means

<xarray.DataArray 'cdr_sea_ice_thickness' (time: 444, rows: 361, columns: 361)> Size: 231MB
array([[[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)
Coordinates:
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
  * time     (time) datetime64[ns] 4kB 1982-01-01 1982-02-01 ... 2018-12-01

In [44]:
# Compute the monthly climatology
monthly_climatology = final_monthly_means.groupby('time.month').mean(dim='time')
monthly_climatology

<xarray.DataArray 'cdr_sea_ice_thickness' (month: 12, rows: 361, columns: 361)> Size: 6MB
array([[[0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        ...,
        [0.        , 0.00018468, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
...
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        [0.        , 0.        , 0.        , ...,        nan,
                nan,        nan],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)
Coordinates:
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [45]:
# Save the climatology and monthly means to NetCDF files for future use
final_monthly_means.to_netcdf('da_sit_monthly_means.nc')
monthly_climatology.to_netcdf('da_sit_clim.nc')

## Part 3: Update CSV with Sea Ice Thickness

In [134]:
# Load the saved DataArrays
da_monthly_means = xr.open_dataarray('da_sit_monthly_means.nc')
da_monthly_means

<xarray.DataArray 'cdr_sea_ice_thickness' (time: 444, rows: 361, columns: 361)> Size: 231MB
[57862524 values with dtype=float32]
Coordinates:
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
  * time     (time) datetime64[ns] 4kB 1982-01-01 1982-02-01 ... 2018-12-01

In [135]:
da_clim = xr.open_dataarray('da_sit_clim.nc')
da_clim

<xarray.DataArray 'cdr_sea_ice_thickness' (month: 12, rows: 361, columns: 361)> Size: 6MB
[1563852 values with dtype=float32]
Coordinates:
  * rows     (rows) float32 1kB -4.5e+06 -4.475e+06 ... 4.5e+06 4.525e+06
  * columns  (columns) float32 1kB -4.525e+06 -4.5e+06 ... 4.475e+06 4.5e+06
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [136]:
# Calculate the mean over the spatial dimensions (rows, columns)
avg_sit = da_monthly_means.mean(dim=['rows', 'columns'])
avg_sit

<xarray.DataArray 'cdr_sea_ice_thickness' (time: 444)> Size: 2kB
array([0.7713997 , 0.91296846, 1.160637  , 1.3964113 , 1.2905824 ,
       0.77134013, 0.4339125 , 0.24814664, 0.21952492, 0.36057857,
       0.54691803, 0.68304366, 0.79261273, 0.94484925, 1.1519578 ,
       1.3643647 , 1.227094  , 0.7605013 , 0.4314174 , 0.2528966 ,
       0.219451  , 0.37248802, 0.54751366, 0.6757466 , 0.70191395,
       0.85427487, 1.1053555 , 1.3370705 , 1.209591  , 0.7250679 ,
       0.42566067, 0.23823632, 0.20835072, 0.32319298, 0.4991263 ,
       0.63523096, 0.7129393 , 0.8330265 , 1.0958304 , 1.357644  ,
       1.2411207 , 0.72641706, 0.39863256, 0.22790316, 0.20081106,
       0.32963476, 0.49382845, 0.62846655, 0.7576866 , 0.87840396,
       1.1247232 , 1.31395   , 1.1953964 , 0.72513396, 0.4136286 ,
       0.2372777 , 0.22486533, 0.35003513, 0.51525927, 0.633935  ,
       0.71771425, 0.9385085 , 1.1185303 , 1.3173112 , 1.202178  ,
       0.7302153 , 0.41497752, 0.22956297, 0.23881973, 0.36016956,
       0.55524296, 0.6810239 , 0.7242177 , 0.8852498 , 1.1475805 ,
       1.3514366 , 1.2128006 , 0.7368466 , 0.4310684 , 0.2588755 ,
       0.24045733, 0.3667192 , 0.54816335, 0.67229503, 0.744822  ,
       0.85134864, 1.0909547 , 1.2630793 , 1.1626555 , 0.73828906,
       0.43524644, 0.24827309, 0.2145473 , 0.35310456, 0.5263737 ,
       0.65560406, 0.6673992 , 0.83313686, 1.0829328 , 1.2414312 ,
...
       0.18906286, 0.29331747, 0.35944313, 0.42817768, 0.56449753,
       0.7749126 , 0.97955066, 0.8546699 , 0.49375683, 0.25896755,
       0.13177188, 0.09687696, 0.17350824, 0.2797288 , 0.33792037,
       0.3962226 , 0.52042806, 0.76473105, 0.9977134 , 0.8501257 ,
       0.4763986 , 0.25051916, 0.11635742, 0.07257104, 0.14063731,
       0.2312074 , 0.31523463, 0.37010905, 0.5490076 , 0.784516  ,
       0.97794753, 0.8429703 , 0.50014687, 0.2618203 , 0.1408506 ,
       0.10835295, 0.19970825, 0.28382522, 0.32243326, 0.41796362,
       0.5289077 , 0.76032376, 0.9523643 , 0.82217544, 0.48300955,
       0.2547967 , 0.13811108, 0.10762776, 0.17002724, 0.28359497,
       0.3241357 , 0.40221876, 0.5560313 , 0.7410693 , 0.9327535 ,
       0.80301905, 0.46194363, 0.25769618, 0.12752733, 0.09356933,
       0.17861338, 0.2551505 , 0.3285819 , 0.36287355, 0.4862009 ,
       0.73323184, 0.9312518 , 0.7582459 , 0.43486252, 0.24032807,
       0.12278125, 0.08521421, 0.14250979, 0.19801039, 0.2719088 ,
       0.3860351 , 0.5037483 , 0.7644734 , 0.9398905 , 0.7862725 ,
       0.4514077 , 0.23920886, 0.1216456 , 0.09691914, 0.16625983,
       0.24244279, 0.27290216, 0.37979376, 0.48192367, 0.7782741 ,
       0.9354771 , 0.7559605 , 0.43657288, 0.2424289 , 0.12060032,
       0.09460527, 0.1263849 , 0.23950243, 0.276466  ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 4kB 1982-01-01 1982-02-01 ... 2018-12-01

In [137]:
df = pd.read_csv('Arctic_domain_mean_monthly_1979_2018.csv')
df

,Date,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
0,1979-01,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,33.341556,273.355237,250.388101,984.633032,15604191.0
1,1979-02,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,33.365164,273.121885,247.071202,983.980418,16378929.0
2,1979-03,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,33.365938,273.088099,252.954138,985.140468,16521089.0
3,1979-04,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,33.350408,273.126062,259.557456,989.314698,15561238.0
4,1979-05,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,33.295768,273.393551,269.375118,984.483658,14085613.0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,2018-08,4.743874,4.882701,308.827087,127.484271,2.129987,0.241098,33.232772,278.534500,279.158164,978.264282,5622998.0
476,2018-09,5.232015,3.581254,283.515381,69.458786,1.957572,0.600476,33.256054,277.887405,274.840139,980.208681,4834953.0
477,2018-10,5.516410,2.440121,257.145101,25.108110,1.784562,0.973192,33.321765,276.161004,269.065448,977.742135,6824601.0
478,2018-11,5.430916,1.440217,220.909249,4.937239,1.320885,0.904150,33.376158,274.805928,259.133665,983.425500,10279618.0


In [138]:
# Prepare the new sea ice thickness column
nan_values = [np.nan] * 36  # fill missing values for 1979-1981
sea_ice_thickness = nan_values + list(avg_sit.values)

# Add the new column to the DataFrame
df.insert(1, 'sit', sea_ice_thickness)  # insert as the first column

df

,Date,sit,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
0,1979-01,NaN,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,33.341556,273.355237,250.388101,984.633032,15604191.0
1,1979-02,NaN,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,33.365164,273.121885,247.071202,983.980418,16378929.0
2,1979-03,NaN,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,33.365938,273.088099,252.954138,985.140468,16521089.0
3,1979-04,NaN,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,33.350408,273.126062,259.557456,989.314698,15561238.0
4,1979-05,NaN,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,33.295768,273.393551,269.375118,984.483658,14085613.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2018-08,0.120600,4.743874,4.882701,308.827087,127.484271,2.129987,0.241098,33.232772,278.534500,279.158164,978.264282,5622998.0
476,2018-09,0.094605,5.232015,3.581254,283.515381,69.458786,1.957572,0.600476,33.256054,277.887405,274.840139,980.208681,4834953.0
477,2018-10,0.126385,5.516410,2.440121,257.145101,25.108110,1.784562,0.973192,33.321765,276.161004,269.065448,977.742135,6824601.0
478,2018-11,0.239502,5.430916,1.440217,220.909249,4.937239,1.320885,0.904150,33.376158,274.805928,259.133665,983.425500,10279618.0


In [139]:
# Fill NaN values with the column mean
df['sit'].fillna(df['sit'].mean(), inplace=True)
df

,Date,sit,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
0,1979-01,0.573670,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,33.341556,273.355237,250.388101,984.633032,15604191.0
1,1979-02,0.573670,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,33.365164,273.121885,247.071202,983.980418,16378929.0
2,1979-03,0.573670,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,33.365938,273.088099,252.954138,985.140468,16521089.0
3,1979-04,0.573670,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,33.350408,273.126062,259.557456,989.314698,15561238.0
4,1979-05,0.573670,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,33.295768,273.393551,269.375118,984.483658,14085613.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2018-08,0.120600,4.743874,4.882701,308.827087,127.484271,2.129987,0.241098,33.232772,278.534500,279.158164,978.264282,5622998.0
476,2018-09,0.094605,5.232015,3.581254,283.515381,69.458786,1.957572,0.600476,33.256054,277.887405,274.840139,980.208681,4834953.0
477,2018-10,0.126385,5.516410,2.440121,257.145101,25.108110,1.784562,0.973192,33.321765,276.161004,269.065448,977.742135,6824601.0
478,2018-11,0.239502,5.430916,1.440217,220.909249,4.937239,1.320885,0.904150,33.376158,274.805928,259.133665,983.425500,10279618.0


In [141]:
# Save the updated DataFrame back to the CSV file
df.to_csv('updated_Arctic_domain_mean_monthly_1979_2018.csv', index=False)

## Part 4: Prepare Data for the LSTM Model

In [142]:
from tempfile import TemporaryFile

# Load data from the updated CSV file
lstm_df = pd.read_csv('updated_Arctic_domain_mean_monthly_1979_2018.csv')
lstm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               480 non-null    object 
 1   sit                480 non-null    float64
 2   wind_10m           480 non-null    float64
 3   specific_humidity  480 non-null    float64
 4   LW_down            480 non-null    float64
 5   SW_down            480 non-null    float64
 6   rainfall           480 non-null    float64
 7   snowfall           480 non-null    float64
 8   sosaline           480 non-null    float64
 9   sst                480 non-null    float64
 10  t2m                480 non-null    float64
 11  surface_pressure   480 non-null    float64
 12  sea_ice_extent     480 non-null    float64
dtypes: float64(12), object(1)
memory usage: 48.9+ KB


In [143]:
# Drop unwanted columns annd keep essential features
lstm_df = lstm_df.drop(['Date'], axis=1)
lstm_df

,sit,wind_10m,specific_humidity,LW_down,SW_down,rainfall,snowfall,sosaline,sst,t2m,surface_pressure,sea_ice_extent
0,0.573670,5.531398,0.811961,186.687054,3.127880,1.009872,0.892319,33.341556,273.355237,250.388101,984.633032,15604191.0
1,0.573670,5.328020,0.688896,174.794571,18.541594,0.920831,0.781347,33.365164,273.121885,247.071202,983.980418,16378929.0
2,0.573670,5.432511,0.916124,190.741933,67.690429,0.983327,0.855266,33.365938,273.088099,252.954138,985.140468,16521089.0
3,0.573670,4.792836,1.272056,212.937925,156.223674,0.890723,0.705203,33.350408,273.126062,259.557456,989.314698,15561238.0
4,0.573670,4.819028,2.239776,253.690478,230.950833,1.201308,0.688723,33.295768,273.393551,269.375118,984.483658,14085613.0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.120600,4.743874,4.882701,308.827087,127.484271,2.129987,0.241098,33.232772,278.534500,279.158164,978.264282,5622998.0
476,0.094605,5.232015,3.581254,283.515381,69.458786,1.957572,0.600476,33.256054,277.887405,274.840139,980.208681,4834953.0
477,0.126385,5.516410,2.440121,257.145101,25.108110,1.784562,0.973192,33.321765,276.161004,269.065448,977.742135,6824601.0
478,0.239502,5.430916,1.440217,220.909249,4.937239,1.320885,0.904150,33.376158,274.805928,259.133665,983.425500,10279618.0


In [144]:
# Convert the DataFrame to a NumPy array
narray = np.array(lstm_df)
print(narray.shape)

(480, 12)


In [145]:
# Separate target variable (last column)
target = narray[:, -1]

In [146]:
# Reshape features for LSTM input
def reshape_features(dataset, timesteps=1):
    print(dataset.shape)
    X = dataset.reshape((int(dataset.shape[0] / timesteps)), timesteps, dataset.shape[1])
    return X

In [147]:
timesteps = 1
narray = reshape_features(narray, timesteps)

# Save the features and target as numpy files
data = TemporaryFile()
np.save('updated_monthly_features.npy', narray)
np.save('updated_monthly_target.npy', target)

(480, 12)
